In [ ]:
import pandas as pd
from zipfile import ZipFile
from transformers import AutoTokenizer, BertForSequenceClassification
import torch


### 0. 上傳要檢查的 csv 檔 sms.csv ###
###    上傳 train_name 產出的模型 zip ###
# === 1. 解壓縮模型 zip（可略過若已解壓）
with ZipFile("/content/model_results.zip", 'r') as zip_ref:
    zip_ref.extractall("results")

# === 2. 載入模型與 tokenizer
dir = "./results/results/final"
tokenizer = AutoTokenizer.from_pretrained(dir)
model = BertForSequenceClassification.from_pretrained(dir)
model.eval()

# === 3. 讀取 CSV 檔案（包含簡訊文字欄位）
df = pd.read_csv("sms.csv")  # ✅ 你要預測的簡訊 CSV 檔案
assert "text" in df.columns, "⚠️ 請確認你的 CSV 包含 text 欄位"

# === 4. 定義預測函式（每筆簡訊）
def predict_person_name(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
    return predicted_label  # 1 = 含人名, 0 = 不含人名

# === 5. 套用到整個欄位，新增一欄結果
df["has_person_name"] = df["text"].apply(predict_person_name)

# ✅ 可選：轉為標籤顯示
df["person_name_result"] = df["has_person_name"].map({1: "✅ 含人名", 0: "❌ 不含人名"})

# === 6. 輸出結果到新的 CSV 檔
df.to_csv("sms_with_name_predictions.csv", index=False)
print("✅ 預測完成，已輸出為 sms_with_name_predictions.csv")